# Libraries

In [1]:
import os
import datetime
import regex as re
import glob
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random
import shutil
from pathlib import Path
from datetime import datetime
import pandas as pd
import html
import numpy as np
import nltk
nltk.download('stopwords')
from string import punctuation
stopwords = set(nltk.corpus.stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\halle.davis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Scraping Data

## Get Links

In [2]:
years = {'year07':"https://arlweb.msha.gov/fatals/indices/FABM2007.asp",
         'year06':"https://arlweb.msha.gov/fatals/indices/FABM2006.asp",
         'year05':"https://arlweb.msha.gov/fatals/indices/FABM2005.asp",
         'year04':"https://arlweb.msha.gov/fatals/indices/FABM2004.HTM",
         'year03':"https://arlweb.msha.gov/fatals/indices/FABM2003.HTM",
         'year02':"https://arlweb.msha.gov/fatals/indices/FABM2002.HTM",
         'year01':"https://arlweb.msha.gov/fatals/indices/FABM2001.HTM",
         'year00':"https://arlweb.msha.gov/fatals/indices/FABM2000.HTM",
         'year99':"https://arlweb.msha.gov/fatals/indices/FABM99.HTM",
         'year98':"https://arlweb.msha.gov/fatals/indices/FABM98.HTM",
         'year97':"https://arlweb.msha.gov/fatals/indices/FABM97.HTM",
         'year96':"https://arlweb.msha.gov/fatals/indices/FABM96.HTM",
         'year95':"https://arlweb.msha.gov/fatals/indices/FABM95.HTM",
        } 

In [ ]:
report_pages = defaultdict(list)
urls = []
start_time = datetime.now()


for year, year_page in years.items() :
    r = requests.get(year_page)
    time.sleep(5 + 10*random.random())
    soup = BeautifulSoup(r.text, 'html.parser')
    links = soup.find_all('a', href=True)
    for link in links:
        if '/FATALS/' and '/FTL' in link['href']:
            urls.append(link.get('href'))
            report_pages[year].append(link.get('href'))
            
end_time = datetime.now()
print("Started At:", start_time, "\nEnded At:", end_time)            

In [ ]:
for year, links in report_pages.items() : 
    print(f"For {year} we have {len(links)}.")
    print(f"The full pull will take for this year will take {round(len(links)*10/3600,2)} hours.")

In [ ]:
urls = [i for i in urls if not ('19' in i)]

## Scrape Text to Files

In [ ]:
## Testing Beautiful Soup Code on Single Link

# 1 - Get text

soup = BeautifulSoup(requests.get("https://arlweb.msha.gov/FATALS/1999/FTL99M01.HTM").content, "html.parser")

In [ ]:
# 2 - Review HTML

print(soup.prettify())

In [ ]:
# 3 - Get Title

print(soup.title.string)

In [ ]:
# 4 - Get Body

print(soup.get_text()) 

In [ ]:
## Run on All Links
## This will take an hour to run!!! (Lots of documents)

url_stub = "https://arlweb.msha.gov/" 
start_time = datetime.now()

total_pages = 0 ## used if we want to limit the report number for performance purposes

os.chdir('C:\\Users\\halle.davis\\Downloads\\reports') ## Halle's work folder path
#os.chdir('/Users/halledavis/Desktop') ## Halle's personal folder path
#os.chdir('') ## Claire's folder path
#os.chdir('') ## Summer's folder path

path = os.getcwd()

for x in urls :
    #if total_pages == 22:   ## used if we want to limit the report number for performance purposes
    #    break 
    name0 = x.replace('/', '')
    name = name0.replace('.HTM', '')
    name = name.replace('.asp', '')
    filename = name+".txt"
    if os.path.exists(filename):
        file = open(filename, "r+")
    else:
        file = open(filename, "w+")
    try:
        soup = BeautifulSoup(requests.get(url_stub+x).content, "html.parser")    
        time.sleep(5 + 10*random.random())
        report = soup.get_text()
        #title = soup.title.string
        file.writelines(report)
        time.sleep(5 + 10*random.random())
        total_pages += 1
    except:
        pass
    
end_time = datetime.now()
print("Started At:", start_time, "\nEnded At:", end_time)

## Download Data to Dataframe

In [3]:
os.chdir('C:\\Users\\halle.davis\\Downloads\\reports') ## Halle's work folder path
#os.chdir('/Users/halledavis/Desktop') ## Halle's personal folder path
#os.chdir('') ## Claire's folder path
#os.chdir('') ## Summer's folder path

file_list = glob.glob(os.path.join(os.getcwd(), "*.txt"))
start_time = datetime.now()

reports = []

for file_path in file_list:
    with open(file_path) as f_input:
        text = (f_input.read())
        file_name = file_path.split("reports")[-1]
        reports.append(
            {
                'filename': file_name,
                'alltext': text
            }
        )
        
data = pd.DataFrame(reports)

end_time = datetime.now()
print("Started At:", start_time, "\nEnded At:", end_time)

Started At: 2023-06-12 14:12:29.454987 
Ended At: 2023-06-12 14:12:29.602730


## Preprocessing

### Get Title

In [4]:
data['title'] = data.alltext.astype(str).str.extract(r': (.*)')

In [5]:
data.head(20)

,filename,alltext,title
0,\FATALS2001FTL01m01.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #1 - Powered Haulage - NV - Gold Unde...
1,\FATALS2001FTL01m02.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #2 - February 9, 2001 - Powered Hau..."
2,\FATALS2001FTL01m03.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #3 - February 16, 2001 - Powered Ha..."
3,\FATALS2001FTL01m04.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #4 - Slip/Fall of Person - NV - Sand...
4,\FATALS2001FTL01m05.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatal #5 - 03/20/2001 - Ignition/Explosion of ...
5,\FATALS2001FTL01m06.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatal #6 - 02/26/2001 - Slip/Fall of Person - ...
6,\FATALS2001FTL01m07.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #7 - Fall of Roof/Back - MT - Platinu...
7,\FATALS2001FTL01m08.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #8 - Machinery - UT - Gold, Lode and..."
8,\FATALS2001FTL01m09.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #9 - Machinery - HI - Stone (CB) - K...
9,\FATALS2001FTL01m10.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #10 - Slip/Fall of Person - AL - Lim...


### Get Cause

In [6]:
### Causes include
#ELECTRICAL - Accidents in which electric current is most directly responsible for the resulting accident.
#ENTRAPMENT - In accidents involving no injuries or nonfatal injuries which are not serious, entrapment of mine workers takes precedence over roof falls, explosives accidents, inundations, etc. If a roof fall results in an entrapment accident, the accident classification is ï¿½Entrapment.
#EXPLODING VESSELS UNDER PRESSURE - These are accidents caused by explosion of air hoses, air tanks, hydraulic lines, hydraulic hoses, and other accidents precipitated by exploding vessels.
#EXPLOSIVES AND BREAKING AGENTS - Accidents involving the detonation of manufactured explosives, Airdox, or Cardox, that can cause flying debris, concussive forces, or fumes.
#FALLING, ROLLING, OR SLIDING ROCK OR MATERIAL OF ANY KIND -Injuries caused directly by falling material require great care in classification. Remember that it is the accident we want to classify. If material was set in motion by machinery, haulage equipment, or hand tools, or while material is being handled or disturbed, etc., charge the force that set the material in motion. For example, where a rock was pushed over a highwall by a dozer and the rock hit another rock which struck and injured a worker - charge the accident to the dozer. Charge the accident to that which most directly caused the resulting accident. Without the dozer, there would have been no resulting accident. This includes accidents caused by improper blocking of equipment under repair or inspection.
#FALL OF FACE, RIB, SIDE OR HIGHWALL - Accidents in this classification include falls of material (from in-place) while barring down or placing props; also pressure bumps and bursts. Since pressure bumps and bursts which cause accidents are infrequent, they are not given a separate category. Not included are accidents in which the motion of machinery or haulage equipment caused the fall either directly or by knocking out support; such accidents are classified as machinery or haulage, whichever is appropriate.
#FALL OF ROOF OR BACK - Underground accidents which include falls while barring down or placing props; also pressure bumps and bursts. Not included are accidents in which the motion of machinery or haulage equipment caused the fall either directly or by knocking out support; such falls are classified as machinery or haulage, whichever is appropriate.
#FIRE - In underground mines, an unplanned fire not extinguished within 10 minutes of discovery; in surface mines and surface areas of underground mines, an unplanned fire not extinguished within 30 minutes of discovery;
#HANDLING MATERIAL - (Lifting, pulling, pushing, shoveling material.) The material may be in bags or boxes, or loose sand, coal, rock, timber, etc. The accident must have been most directly caused by handling material.
#HAND TOOLS - Accidents related to non-powered tools when being used as hand tools. Do not include electric tools or air-powered tools.
#HOISTING - Damage to hoisting equipment in a shaft or slope which endangers an individual or interferes with use of the equipment for more than 30 minutes. Hoisting may also be the classification where a victim was injured by hoisting equipment but there was no damage to the equipment. Accidents involving cages, skips, buckets, or elevators. The accident results from the action, motion, or failure of the hoisting equipment or mechanism. Included is equipment such as derricks and cranes only when used in shaft sinking; suspended work platforms in shafts; mine cars being lowered or raised by hoisting equipment on slopes or inclines; a skip squeezed between timbers resulting in an accident; or an ore bucket tipped for any reason causing an accident.
#IGNITION OR EXPLOSION OF GAS OR DUST - Accidents resulting as a consequence of the ignition or explosion of gas or dust. Included are exploding gasoline vapors, space heaters, or furnaces.
#Methane Ignition - A methane ignition occurs when methane burns without producing destructive forces. Damage resulting from an ignition is limited to that caused by flame and heat. Personnel in the immediate vicinity of an ignition may be burned and line brattice or other materials in close proximity may be discolored, melted or burned. Ignitions generally involve small quantities of methane and are usually confined to a small area; however, in the case of methane roof layering, flame spread may be more extensive.
#Methane Explosion - A methane explosion occurs when methane is ignited and burns violently. The flame of the explosion accelerates rapidly, heating the environment and causing destructive forces. Evidence of the destructive forces may be manifest on victims, equipment, structures, etc. Witnesses to an explosion may hear the noise generated by the resulting sound pressure wave.
#IMPOUNDMENT - An unstable condition at an impoundment, refuse pile, or culm bank which requires emergency action in order to prevent failure, or which causes individuals to evacuate an area. Also the failure of an impoundment, refuse pile, or culm bank.
#INUNDATION - An unplanned inundation of a mine by a liquid or gas. The mine may be either a surface or underground operation.
#MACHINERY - Accidents that result from the action or motion of machinery or from failure of component parts. Included are all electric and air-powered tools and mining machinery such as drills, tuggers, slushers, draglines, power shovels, loading machines, compressors, etc. Include derricks and cranes except when they are used in shaft sinking (see HOISTING) or mobile cranes traveling with a load (see POWERED HAULAGE).
#NON-POWERED HAULAGE - Accidents related to motion of non-powered haulage equipment. Included are accidents involving wheelbarrows, manually pushed mine cars and trucks, etc.
#POWERED HAULAGE - Haulage includes motors and rail cars, conveyors, belt feeders, longwall conveyors, bucket elevators, vertical manlifts, self-loading scrapers or pans, shuttle cars, haulage trucks, front-end loaders, load-haul- dumps, forklifts, cherry pickers, mobile cranes if traveling with a load, etc. The accident is caused by the motion of the haulage unit. Include accidents that are caused by an energized or moving unit or failure of component parts. If a car dropper suffers an injury as a result of falling from a moving car, charge the accident to haulage.
#SLIP OR FALL OF PERSON - Includes slips or falls from an elevated position or at the same level while getting on or off machinery or haulage equipment that is not moving. Also includes slips or falls while servicing or repairing equipment or machinery. Includes stepping in a hole.
#STEPPING OR KNEELING ON OBJECT - Accidents are classified in this category only where the object stepped or kneeled on contributed most directly to the accident.
#STRIKING OR BUMPING - This classification is restricted to those accidents in which an individual, while moving about, strikes or bumps an object but is not handling material, using hand tools, or operating equipment.
#OTHER - Accidents not elsewhere classified. This is a last resort category.

In [7]:
#Cause Keywords

electric = ['electric', 'electrical']
entrapment = ['entrapment']
explodingvessels = ['vessels', 'pressure']
explosives = ['explosives', 'agents']
fallingmaterial = ['material', 'rock']
fallofface = ['face', 'rib', 'side', 'highwall']
fallofroof = ['roof', 'back']
fire = ['fire']
handlingmaterial = ['handling']
handtools = ['hand', 'handtools']
hoisting = ['hoist', 'hositing']
ignition = ['ignition', 'gas', 'dust', 'ignition/explosion', 'methane']
impoundment = ['impoundment']
inundation = ['inundation']
machinery = ['machinery']
nonpoweredhaulage = ['non-powered']
poweredhaulage = ['powered']
slipfall = ['slip', 'fall', 'slip/fall', '/fall']
stepping = ['stepping', 'kneeling']
striking = ['striking', 'bumping']
other = ['other']

In [8]:
def causeclassifier(sentence):
    if any(word in electric for word in sentence.lower().split()):
        return 'Electric'
    elif any(word in entrapment for word in sentence.lower().split()):
        return 'Entrapment'
    elif any(word in explodingvessels for word in sentence.lower().split()):
        return 'Exploding Vessels'
    elif any(word in explosives for word in sentence.lower().split()):
        return 'Explosives'
    elif any(word in fallingmaterial for word in sentence.lower().split()):
        return 'Falling Material'
    elif any(word in fallofface for word in sentence.lower().split()):
        return 'Fall of Face'
    elif any(word in fallofroof for word in sentence.lower().split()):
        return 'Fall of Roof'
    elif any(word in fire for word in sentence.lower().split()):
        return 'Fire'
    elif any(word in handlingmaterial for word in sentence.lower().split()):
        return 'Handling Material'
    elif any(word in handtools for word in sentence.lower().split()):
        return 'Hand Tools'
    elif any(word in hoisting for word in sentence.lower().split()):
        return 'Hoisting'
    elif any(word in ignition for word in sentence.lower().split()):
        return 'Ignition/Explosion'
    elif any(word in inundation for word in sentence.lower().split()):
        return 'Inundation'
    elif any(word in machinery for word in sentence.lower().split()):
        return 'Machinery'
    elif any(word in nonpoweredhaulage for word in sentence.lower().split()):
        return 'Non-Powered Haulage'
    elif any(word in poweredhaulage for word in sentence.lower().split()):
        return 'Powered Haulage'
    elif any(word in slipfall for word in sentence.lower().split()):
        return 'Slip/Fall'
    elif any(word in stepping for word in sentence.lower().split()):
        return 'Stepping'
    elif any(word in striking for word in sentence.lower().split()):
        return 'Striking'
    elif any(word in other for word in sentence.lower().split()):
        return 'Other'
    else:
        return 'Other'

In [9]:
data['cause'] = data['title'].astype(str).apply(causeclassifier)    

In [10]:
data.head(20)

,filename,alltext,title,cause
0,\FATALS2001FTL01m01.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #1 - Powered Haulage - NV - Gold Unde...,Powered Haulage
1,\FATALS2001FTL01m02.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #2 - February 9, 2001 - Powered Hau...",Powered Haulage
2,\FATALS2001FTL01m03.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #3 - February 16, 2001 - Powered Ha...",Powered Haulage
3,\FATALS2001FTL01m04.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #4 - Slip/Fall of Person - NV - Sand...,Slip/Fall
4,\FATALS2001FTL01m05.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatal #5 - 03/20/2001 - Ignition/Explosion of ...,Ignition/Explosion
5,\FATALS2001FTL01m06.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatal #6 - 02/26/2001 - Slip/Fall of Person - ...,Slip/Fall
6,\FATALS2001FTL01m07.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #7 - Fall of Roof/Back - MT - Platinu...,Slip/Fall
7,\FATALS2001FTL01m08.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #8 - Machinery - UT - Gold, Lode and...",Machinery
8,\FATALS2001FTL01m09.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #9 - Machinery - HI - Stone (CB) - K...,Machinery
9,\FATALS2001FTL01m10.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #10 - Slip/Fall of Person - AL - Lim...,Slip/Fall


### Get Year

In [11]:
# Get year

data['year']  = data.alltext.astype(str).str.extract(r'(200\d)')

In [12]:
data.tail(20)

,filename,alltext,title,cause,year
182,\FATALS2007FTL07m13.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #13 - June 13, 2007",Other,2007
183,\FATALS2007FTL07m14.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #33 - September 14, 2007 - (Chargebac...",Electric,2007
184,\FATALS2007FTL07m15.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #15 - July 18, 2007",Other,2007
185,\FATALS2007FTL07m1617.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #16 and #17 - July 19, 2007 - Machin...",Machinery,2007
186,\FATALS2007FTL07m18.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #18 - July 24, 2007",Other,2007
187,\FATALS2007FTL07m19.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #19 - July 29, 2007",Other,2007
188,\FATALS2007FTL07m20.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #20 - July 30, 2007",Other,2007
189,\FATALS2007FTL07m21.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #21 - August 28, 2007 - Fall of Roof...",Fall of Roof,2007
190,\FATALS2007FTL07m22.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #22 - August 29, 2007 - Fall of Pers...",Slip/Fall,2007
191,\FATALS2007FTL07m23.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal/Nonmetal M...,"Fatality #23 - September 7, 2007 - Machinery ...",Machinery,2007


### Clean & Tokenize Text

In [13]:
def tokenize(text):
    return re.findall(r'[\w-]*\p{L}[\w-]*', text)

In [14]:
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#"}

def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))

In [15]:
def remove_stop(tokens):
    return [t for t in tokens if t.lower() not in stopwords]

In [16]:
pipeline = [str.lower, tokenize, remove_stop]
def prepare(text, pipeline):
    tokens = text
    for transform in pipeline:
        tokens = transform(tokens)
    return tokens

In [17]:
def clean(text):
    # convert html escapes like &amp; to characters.
    text = html.unescape(text)
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text) # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    # numbers
    text = re.sub(r'\d+', '', text)
    return text.strip()

In [18]:
data['tokens'] = data['alltext'].apply(prepare, pipeline=pipeline)

In [19]:
data["cleantext"] = data["alltext"].apply(clean)
data["cleantext"] = data["cleantext"].apply(str.lower)
data["cleantext"] = data["cleantext"].apply(remove_punctuation)

In [20]:
data.head(20)

,filename,alltext,title,cause,year,tokens,cleantext
0,\FATALS2001FTL01m01.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #1 - Powered Haulage - NV - Gold Unde...,Powered Haulage,2001,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...
1,\FATALS2001FTL01m02.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #2 - February 9, 2001 - Powered Hau...",Powered Haulage,2001,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...
2,\FATALS2001FTL01m03.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #3 - February 16, 2001 - Powered Ha...",Powered Haulage,2001,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...
3,\FATALS2001FTL01m04.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #4 - Slip/Fall of Person - NV - Sand...,Slip/Fall,2001,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...
4,\FATALS2001FTL01m05.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatal #5 - 03/20/2001 - Ignition/Explosion of ...,Ignition/Explosion,2001,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...
5,\FATALS2001FTL01m06.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatal #6 - 02/26/2001 - Slip/Fall of Person - ...,Slip/Fall,2001,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...
6,\FATALS2001FTL01m07.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #7 - Fall of Roof/Back - MT - Platinu...,Slip/Fall,2001,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...
7,\FATALS2001FTL01m08.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #8 - Machinery - UT - Gold, Lode and...",Machinery,2001,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...
8,\FATALS2001FTL01m09.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #9 - Machinery - HI - Stone (CB) - K...,Machinery,2001,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...
9,\FATALS2001FTL01m10.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #10 - Slip/Fall of Person - AL - Lim...,Slip/Fall,2001,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...


### Get Text by Category

In [21]:
# Get Overview

data['overview']  = data.cleantext.astype(str).str.extract(r'overview(.*) general info')

In [22]:
# Get General Information

data['geninfo']  = data.cleantext.astype(str).str.extract(r'information(.*)description of') #sometimes its "the accident" and sometimes its just accident

In [23]:
# Description of Accident

data['description']  = data.cleantext.astype(str).str.extract(r'description of(.*)investigation of') #sometimes its "the accident" and sometimes its just accident

In [24]:
# Other Info
# contains information from the investigation, root cause, and discussion segments
# did not separate because headers vary in order and presence, so was difficult to parse out those exact sections

data['otherinfo']  = data.cleantext.astype(str).str.extract(r'investigation of(.*)conclusion') #sometimes its "the accident" and sometimes its just accident



In [25]:
# Failed Attempt to Pull Out Investigation Only -- doesn't take into consideration ordering differences
# investigation = []

#for value in data['cleantext']:
#    if 'root cause analysis' in value:
#        investigation.append(re.findall(r'investigation of(.*)root cause analysis', value))
#    else:
#        investigation.append(re.findall(r'investigation of(.*) discussion', value))

# data['investigation'] = investigation

In [26]:
data['conclusion']  = data.cleantext.astype(str).str.extract(r'conclusion(.*)enforcement')

In [27]:
data['enforcement']  = data.cleantext.astype(str).str.extract(r'enforcement action(.*)appendix')

In [36]:
data['overview'] = data['overview'].astype(str)
data['overview_tokens'] = data['overview'].apply(prepare, pipeline=pipeline)

data['geninfo'] = data['geninfo'].astype(str)
data['geninfo_tokens'] = data['geninfo'].apply(prepare, pipeline=pipeline)

data['otherinfo'] = data['otherinfo'].astype(str)
data['otherinfo_tokens'] = data['otherinfo'].apply(prepare, pipeline=pipeline)

data['conclusion'] = data['conclusion'].astype(str)
data['conclusion_tokens'] = data['conclusion'].apply(prepare, pipeline=pipeline)

data['enforcement'] = data['enforcement'].astype(str)
data['enforcement_tokens'] = data['enforcement'].apply(prepare, pipeline=pipeline)

In [37]:
data.head(20)

,filename,alltext,title,cause,year,tokens,cleantext,overview,geninfo,description,otherinfo,conclusion,enforcement,overview_tokens,geninfo_tokens,otherinfo_tokens,conclusion_tokens,enforcement_tokens
0,\FATALS2001FTL01m01.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #1 - Powered Haulage - NV - Gold Unde...,Powered Haulage,2001,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...,thomas j scott miner age was fatally injure...,the carlin mine a multilevel underground gold...,the accident thomas scott victim reported for...,the accident msha was notified at am on the ...,the cause of the accident was the failure to ...,s order no was issued on january under the ...,"[thomas, j, scott, miner, age, fatally, injure...","[carlin, mine, multilevel, underground, gold, ...","[accident, msha, notified, day, accident, tele...","[cause, accident, failure, maintain, berm, bum...","[order, issued, january, provisions, section, ..."
1,\FATALS2001FTL01m02.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #2 - February 9, 2001 - Powered Hau...",Powered Haulage,2001,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...,on february dustin l hollibaugh plant oiler...,portable # a surface sand and gravel mine own...,the accident on the day of the accident dusti...,the accident msha was notified of the acciden...,the accident was caused by the failure to dee...,s tommer construction co inc order no was iss...,"[february, dustin, l, hollibaugh, plant, oiler...","[portable, surface, sand, gravel, mine, owned,...","[accident, msha, notified, accident, february,...","[accident, caused, failure, deenergize, power,...","[tommer, construction, co, inc, order, issued,..."
2,\FATALS2001FTL01m03.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,"Fatality #3 - February 16, 2001 - Powered Ha...",Powered Haulage,2001,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...,on february jon r skinner mechanic age was...,south area a multilevel open pit gold mine ow...,the accident on the day of the accident jon s...,the accident msha was notified at pm on febr...,the cause of the accident was the failure to ...,s order no was issued on february under the...,"[february, jon, r, skinner, mechanic, age, fat...","[south, area, multilevel, open, pit, gold, min...","[accident, msha, notified, pm, february, telep...","[cause, accident, failure, parking, brake, set...","[order, issued, february, provisions, section,..."
3,\FATALS2001FTL01m04.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatality #4 - Slip/Fall of Person - NV - Sand...,Slip/Fall,2001,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...,wade w pearson contract welder age was fatal...,paiute aggregates a surface sand and gravel m...,the accident on the day of the accident wade ...,the accident msha was notified at pm on the ...,the accident was caused by the failure to ens...,s paiute aggregates inc order no was issued o...,"[wade, w, pearson, contract, welder, age, fata...","[paiute, aggregates, surface, sand, gravel, mi...","[accident, msha, notified, pm, day, accident, ...","[accident, caused, failure, ensure, opening, g...","[paiute, aggregates, inc, order, issued, febru..."
4,\FATALS2001FTL01m05.txt,\n\n\n\n\n\n\n\n\n\n\nMSHA - Metal and Nonmet...,Fatal #5 - 03/20/2001 - Ignition/Explosion of ...,Ignition/Explosion,2001,"[msha, metal, nonmetal, mine, fatal, accident,...",msha metal and nonmetal mine fatal accident in...,on march norman jenkins road construction f...,plant no a portable crushing plant owned and...,the accident on march the day before the ac...,the accident msha was notified at pm on marc...,the cause of the accident was the introductio...,s order no was issued on march under the pr...,"[march, norman, jenkins, road, construction, f...","[plant, portable, crushin

## EDA

In [39]:
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :    
    counter = Counter()
    tokens.map(counter.update)
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    
    counter_df = pd.DataFrame.from_dict(counter, orient='index').reset_index()
    
    num_tokens = sum(freq_df['freq'])
    num_unique_tokens = freq_df.shape[0]
    lexical_diversity = num_unique_tokens / num_tokens
    num_characters = sum((counter_df['index'].str.len()) * counter_df[0])
    
    if verbose :        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
        print(f"The top 5 most common words are")
        print(counter.most_common(5))
        
    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters])

In [40]:
descriptive_stats(data["tokens"])

There are 282320 tokens in the data.
There are 12572 unique tokens in the data.
There are 1830443 characters in the data.
The lexical diversity is 0.045 in the data.
The top 5 most common words are
[('mine', 5887), ('accident', 4258), ('safety', 3916), ('health', 2826), ('msha', 2105)]


[282320, 12572, 0.044531028620005665, 1830443]

## Modeling